# Homework 5 - Part 1
## MET CS 688
### Maryam Asghari

In [2]:
# Dependencies

import requests  
#The requests module allows us to send HTTP requests using Python

from bs4 import BeautifulSoup  
#This line is used to import the Beautiful Soup library into the application.

# import Pandas Library
import pandas as pd  

# import RE Library
import re 
#Regular expression operations Library the functions in this module let us check 
#if a particular string matches a given regular expression

#Using Splinter to automate browser actions
from splinter import Browser

### Getting 50 titles from bing news with keywords: "Fitness" and "Wearables"

In [2]:
#choosing chrom driver 
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [84]:
query = '\"Fitness\" and \"Wearables\" '.lower()  
url = f'https://www.bing.com/news/search?q={query}&form=QBNH'

In [102]:
# visiting the Url in the our browser 
browser.visit(url)

In [103]:
html = browser.html

In [104]:
#Creat a Beautiful Soup object and the object returned is assigned to the soup variable.
soup = BeautifulSoup(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [106]:
news = soup.find_all('div', class_='t_s', limit = 50)
print(f'Number of scraped news: {len(news)}') #check out if we have 20

Number of scraped news: 50


In [107]:
titles = [n.find('a',class_ = 'title').text for  n  in news]   #.text :Extract the text of the tittle
titles

['Smart Wearables for Sports and Fitness Market by Size, Growth, Opportunity and Forecast to 2027',
 "Health and fitness wearables have boomed during the pandemic - and they're changing the way we eat, sleep, exercise and drink alcohol",
 'Wearables shipments surged in 2020, thanks to COVID-19',
 'IDC: Wearables Market Grew 28.4 Percent in 2020',
 'Researchers discover new approach to thermal protection in outdoor wearables',
 'Future Amazfit wearables could have ECG and blood pressure monitoring',
 'Apple continues to dominate wearables market with Apple Watch and AirPods, according to new IDC data',
 '4 Stocks to Watch Amid Booming Demand for Wearables',
 'Style-deprived fitness bands have helped cheap, ugly smartwatches to flourish',
 'IDC: Pandemic shifted consumer spending to wearables market',
 'The Zacks Analyst Blog Highlights: Apple, Google, Amazon and Gramin',
 "Mother's day: Indulge your mom with the most popular Health and Fitness apps on HUAWEI AppGallery",
 'Huawei Talkba

In [20]:
news_df = pd.DataFrame({'title': titles} )
news_df.head()

,title
0,Smart Wearables for Sports and Fitness Market ...
1,Health and fitness wearables have boomed durin...
2,"Wearables shipments surged in 2020, thanks to ..."
3,IDC: Wearables Market Grew 28.4 Percent in 2020
4,Researchers discover new approach to thermal p...


In [37]:
import numpy as np
import pandas as pd
from afinn import Afinn

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import string
import re 


## Preprocessing the titles

In [40]:
stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    # Lower case
    #x = x.lower()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s\-]', ' ', x)
   # x = re.sub(r'[^\P{P}\-]+', ' ', x)
    
    
    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 

%time news_df['title_clean'] = news_df['title'].apply(preprocess)
news_df[['title_clean','title']]

CPU times: user 3.14 ms, sys: 93 µs, total: 3.23 ms
Wall time: 3.23 ms


,title_clean,title
0,Smart Wearables Sports Fitness Market Size Gro...,Smart Wearables for Sports and Fitness Market ...
1,Health fitness wearable boomed pandemic - chan...,Health and fitness wearables have boomed durin...
2,Wearables shipment surged thanks COVID-,"Wearables shipments surged in 2020, thanks to ..."
3,IDC Wearables Market Grew Percent,IDC: Wearables Market Grew 28.4 Percent in 2020
4,Researchers discover new approach thermal prot...,Researchers discover new approach to thermal p...
5,Future Amazfit wearable could ECG blood pressu...,Future Amazfit wearables could have ECG and bl...
6,Apple continues dominate wearable market Apple...,Apple continues to dominate wearables market w...
7,Stocks Watch Amid Booming Demand Wearables,4 Stocks to Watch Amid Booming Demand for Wear...
8,Style-deprived fitness band helped cheap ugly ...,Style-deprived fitness bands have helped cheap...
9,IDC Pandemic shifted consumer spending wearabl...,IDC: Pandemic shifted consumer spending to wea...


## Using Afinn Sentiment Analyzer


In [41]:
# Sentiment analysis with AFINN
afinn = Afinn(emoticons=True)
afinn_scores = [afinn.score(text) for text in news_df.title_clean]
news_df['afinn_sentimen'] = afinn_scores

news_df[['title', 'afinn_sentimen']]

,title,afinn_sentimen
0,Smart Wearables for Sports and Fitness Market ...,6.0
1,Health and fitness wearables have boomed durin...,1.0
2,"Wearables shipments surged in 2020, thanks to ...",2.0
3,IDC: Wearables Market Grew 28.4 Percent in 2020,0.0
4,Researchers discover new approach to thermal p...,0.0
5,Future Amazfit wearables could have ECG and bl...,-1.0
6,Apple continues to dominate wearables market w...,0.0
7,4 Stocks to Watch Amid Booming Demand for Wear...,-1.0
8,Style-deprived fitness bands have helped cheap...,-2.0
9,IDC: Pandemic shifted consumer spending to wea...,0.0


## Using NLTK’s Pre-Trained Sentiment Analyzer

NLTK already has a built-in, pretrained sentiment analyzer called VADER (Valence Aware Dictionary and sEntiment Reasoner).

Since VADER is pretrained, you can get results more quickly than with many other analyzers. However, VADER is best suited for language used in social media, like short sentences with some slang and abbreviations. It’s less accurate when rating longer, structured sentences, but it’s often a good launching point.

You’ll get back a dictionary of different scores. The negative, neutral, and positive scores are related: They all add up to 1 and can’t be negative. The compound score is calculated differently. It’s not just an average, and it can range from -1 to 1.

In [23]:
 from nltk.sentiment import SentimentIntensityAnalyzer

In [42]:
 from nltk.sentiment import SentimentIntensityAnalyzer
# Sentiment analysis using nltk_SentimentIntensityAnalyzer

Sent = SentimentIntensityAnalyzer()
SIA = [Sent.polarity_scores(text).get('compound') for text in news_df.title_clean]
news_df['nltk_sentiment'] = SIA 
news_df[['title', 'afinn_sentimen','nltk_sentiment']]

,title,afinn_sentimen,nltk_sentiment
0,Smart Wearables for Sports and Fitness Market ...,6.0,0.8481
1,Health and fitness wearables have boomed durin...,1.0,0.2732
2,"Wearables shipments surged in 2020, thanks to ...",2.0,0.4404
3,IDC: Wearables Market Grew 28.4 Percent in 2020,0.0,0.0000
4,Researchers discover new approach to thermal p...,0.0,0.0000
5,Future Amazfit wearables could have ECG and bl...,-1.0,-0.2960
6,Apple continues to dominate wearables market w...,0.0,-0.1280
7,4 Stocks to Watch Amid Booming Demand for Wear...,-1.0,-0.1280
8,Style-deprived fitness bands have helped cheap...,-2.0,-0.2960
9,IDC: Pandemic shifted consumer spending to wea...,0.0,0.0000


## Using TextBlob Sentiment Analyzer

Another library for nlp like NLTK 

The sentiment function of textblob returns two properties, polarity, and subjectivity.

Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement. 

In [1]:
from textblob import TextBlob

In [43]:
from textblob import TextBlob
SIA = [TextBlob(text).sentiment.polarity for text in news_df.title_clean]
news_df['TextBlob_sentiment'] = SIA 
news_df[['title', 'afinn_sentimen','nltk_sentiment','TextBlob_sentiment']]

,title,afinn_sentimen,nltk_sentiment,TextBlob_sentiment
0,Smart Wearables for Sports and Fitness Market ...,6.0,0.8481,0.214286
1,Health and fitness wearables have boomed durin...,1.0,0.2732,0.000000
2,"Wearables shipments surged in 2020, thanks to ...",2.0,0.4404,0.200000
3,IDC: Wearables Market Grew 28.4 Percent in 2020,0.0,0.0000,0.000000
4,Researchers discover new approach to thermal p...,0.0,0.0000,0.136364
5,Future Amazfit wearables could have ECG and bl...,-1.0,-0.2960,0.000000
6,Apple continues to dominate wearables market w...,0.0,-0.1280,0.136364
7,4 Stocks to Watch Amid Booming Demand for Wear...,-1.0,-0.1280,0.000000
8,Style-deprived fitness bands have helped cheap...,-2.0,-0.2960,-0.150000
9,IDC: Pandemic shifted consumer spending to wea...,0.0,0.0000,0.000000


## USING TRANSFORMERS PIPELINE SENTIMENT Analyzer
For the task of getting the sentiment of any sentence or document, we use the ‘sentiment-analysis’ pipeline, which enables us to use the recently released **DistilBert model** fine tuned on SST-2 dataset (a benchmark dataset for sentiment analysis).

The given code outputs a dictionary with the following keys:-

‘label’: The sentiment of the statement represented by its respective label.
‘score’: The confidence with which the model has predicted the given label.

In [27]:
from transformers import pipeline
classify = pipeline('sentiment-analysis')

In [28]:
from transformers import pipeline
classify = pipeline('sentiment-analysis')

def get_score(text):
    sent = classify(text)
    score = sent[0].get('score')
    return score

def get_label (text):
    sent = classify(text)
    label = sent[0].get('label')
    return label

In [29]:
transf_sc = [get_score(text) for text in news_df.title]
news_df['transformers_score'] = transf_sc
transf_l = [get_label(text) for text in news_df.title]
news_df['transformers_label'] = transf_l
news_df[['title', 'afinn_sentimen','nltk_sentiment','TextBlob_sentiment','transformers_score','transformers_label']]

,title,afinn_sentimen,nltk_sentiment,TextBlob_sentiment,transformers_score,transformers_label
0,Smart Wearables for Sports and Fitness Market ...,6.0,0.8481,0.214286,0.998387,POSITIVE
1,Health and fitness wearables have boomed durin...,1.0,0.2732,0.000000,0.967633,POSITIVE
2,"Wearables shipments surged in 2020, thanks to ...",2.0,0.4404,0.200000,0.992425,POSITIVE
3,IDC: Wearables Market Grew 28.4 Percent in 2020,0.0,0.0000,0.000000,0.794919,POSITIVE
4,Researchers discover new approach to thermal p...,0.0,0.0000,0.136364,0.994138,POSITIVE
5,Future Amazfit wearables could have ECG and bl...,-1.0,-0.2960,0.000000,0.995878,NEGATIVE
6,Apple continues to dominate wearables market w...,0.0,-0.1280,0.136364,0.985247,NEGATIVE
7,4 Stocks to Watch Amid Booming Demand for Wear...,-1.0,-0.1280,0.000000,0.906597,POSITIVE
8,Style-deprived fitness bands have helped cheap...,-2.0,-0.2960,-0.150000,0.998153,NEGATIVE
9,IDC: Pandemic shifted consumer spending to wea...,0.0,0.0000,0.000000,0.987104,NEGATIVE


In [35]:
news_df.to_excel('titles_50.xlsx', index = False)

In [ ]:
# Next part: Using Google colab to use it's GPU 